In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [16]:
import time
import random
import requests

import signaturehelper

In [ ]:
# !pip install requests==2.21.0 

In [2]:
!pip show requests

Name: requests
Version: 2.21.0
Summary: Python HTTP for Humans.
Home-page: http://python-requests.org
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: c:\users\epsilon\dev\anaconda\envs\kera\lib\site-packages
Requires: idna, chardet, certifi, urllib3
Required-by: tweepy, tensorboard, requests-oauthlib, pandas-profiling, jupyterlab-server, folium


In [28]:
get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID)

{'Content-Type': 'application/json; charset=UTF-8',
 'X-Timestamp': '1612338453856',
 'X-API-KEY': '01000000002c6573aff319a7fb49a48153878c0e3559a4cddf9da1a4e404cb94b9716d9b85',
 'X-Customer': '1930249',
 'X-Signature': b'RyUG4nE9gvuWgv0HR1vbX+DIBoo/MZt9G2y50vxt7rM='}

In [52]:
# it has bug now. they don't match with real keys
keydict = {}
with open('./keys.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        keyname = line.split('=')[0]
        key = line.split('=',maxsplit=1)[1].strip()
        keydict[keyname] = key
    

API_KEY = keydict['API_KEY']
SECRET_KEY = keydict['SECRET_KEY']
CUSTOMER_ID = int(keydict['CUSTOMER_ID'])

In [57]:
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {
            'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID), 
            'X-Signature': signature
           }

BASE_URL = 'https://api.naver.com'


# API_KEY='01000000002c6573aff319a7fb49a48153878c0e3559a4cddf9da1a4e404cb94b9716d9b85'
# SECRET_KEY='AQAAAAAsZXOv8xmn+0mkgVOHjA41adczLrAq2j84OKjr0N/NNA=='
# CUSTOMER_ID=1930249
uri = '/keywordstool'
method = 'GET'
payload={
        'hintKeywords':'쌍화차',
        'showDetail' :1
        }

r = requests.get(BASE_URL + uri, params=payload, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
# json = payload #if you use 

# print("response status_code = {}".format(r.status_code) )
# print("response body = {}".format(r.json() )  )

keywords = r.json()['keywordList'][:100]

del r

#### response json Structure
{'keyswordList':  [{...},{...},, .    ]  }   
{
    "relKeyword": "쌍화탕",
    "monthlyPcQcCnt": ">10",
    "monthlyMobileQcCnt": ">10",
    "monthlyAvePcClkCnt": ">10",
    "monthlyAveMobileClkCnt": ">10",
    "monthlyAvePcCtr": ">10",
    "monthlyAveMobileCtr": ">10",
    "plAvgDepth": ">10",
    "compIdx": "high"
}

In [65]:
keywords = pd.DataFrame(keywords)

In [70]:
pd.set_option('display.max_rows',200)

- 월간 검색수 : 네이버를 이용한 사용자가 PC 및 모바일 에서 해당 키워드를 검색한 횟수
- 월평균 클릭수 : 최근 한 달 간 사용자가 해당 키워드를 검색했을 때, 통합검색 영역에 노출된 광고가 받은 평균 클릭수 입니다 
- 월평균 클릭률 
- 경쟁 정도 : 높음 / 중간 / 낮음


In [73]:
keywords.columns = ['relkeyword','sch','mob_sch','click','mob_click','clk_r','mob_clk_r','num_ads','comp']

In [80]:
comp_dic={'높음':2,'중간':1,'낮음':0}

keywords.comp = keywords.comp.apply(lambda x : comp_dic[x])
keywords.sch = keywords.sch.astype('str').str.replace('< ','').astype('int')
keywords.mob_sch = keywords.mob_sch.astype('str').str.replace('< ','').astype('int')

In [115]:
# keywords.sort_values(by=['sch','clk_r'],ascending=False)

,relkeyword,sch,mob_sch,click,mob_click,clk_r,mob_clk_r,num_ads,comp
37,생수,17700,89700,53.9,771.5,0.33,0.92,15,2
41,컵라면,9710,38400,35.1,275.2,0.39,0.77,15,2
0,쌍화차,7120,56900,51.7,1589.8,0.78,2.85,15,2
10,우엉차,5460,29600,17.4,552.1,0.36,1.99,15,2
1,생강차,4770,25100,16.7,432.6,0.37,1.76,15,2
23,율무차,3970,18000,11.7,238.7,0.32,1.39,15,2
11,둥굴레차,3840,11600,10.7,219.6,0.31,2.02,15,2
8,핫초코,3380,15200,3.7,15.0,0.12,0.11,15,2
18,옥수수수염차,2950,15700,8.4,26.0,0.31,0.18,15,2
3,대추차,2340,14300,9.1,241.9,0.42,1.76,15,2


In [87]:
keywords.sort_values(by=['sch'])

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
### 

In [ ]:
# BusinessChannel Usage Sample

uri = '/ncc/channels'
method = 'GET'
r = requests.get(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))


# Adgroup Usage Sample

# 1. GET adgroup Usage Sample

uri = '/ncc/adgroups'
method = 'GET'
r = requests.get(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))
target_adgroup = r.json()[0]

# 2. CREATE adgroup Usage Sample

uri = '/ncc/adgroups'
method = 'POST'
payload = {'name': 'TEST#' + str(random.randrange(1000, 9999)), 'nccCampaignId' : target_adgroup['nccCampaignId'], 'pcChannelId' : target_adgroup['pcChannelId'], 'mobileChannelId': target_adgroup['mobileChannelId']}
r = requests.post(BASE_URL + uri, json=payload, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

created_adgroup = r.json()

# 3. UPDATE Adgroup Usage Sample

uri = '/ncc/adgroups/' + created_adgroup['nccAdgroupId']
method = 'PUT'
created_adgroup['userLock'] = 0
r = requests.put(BASE_URL + uri, params={'fields': 'userLock'}, json=created_adgroup, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# 4. DELETE Adgroup

uri = '/ncc/adgroups/' + created_adgroup['nccAdgroupId']
method = 'DELETE'
r = requests.delete(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.content))

# AdKeyword Usage Sample

# 1. CREATE AdKeyword

uri = '/ncc/keywords'
method = 'POST'
r = requests.post(BASE_URL + uri, params={'nccAdgroupId': created_adgroup['nccAdgroupId']}, json=[{'keyword': 'hello'}], headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

created_adkeyword = r.json()[0]

# 2. GET AdKeyword

uri = '/ncc/keywords'
method = 'GET'
r = requests.get(BASE_URL + uri, params={'nccAdgroupId': created_adgroup['nccAdgroupId']}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# 3. UPDATE AdKeyword

uri = '/ncc/keywords'
method = 'PUT'
created_adkeyword['userLock'] = 0
r = requests.put(BASE_URL + uri, params={'fields': 'userLock'}, json=[created_adkeyword], headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))


# 4. GET and UPDATE AdKeyword (BidAmt)

uri = '/ncc/keywords/' + created_adkeyword['nccKeywordId']
method = 'GET'
r = requests.get(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

retrieved_adkeyword = r.json()

uri = '/ncc/keywords'
method = 'PUT'
retrieved_adkeyword['bidAmt'] = 300
retrieved_adkeyword['useGroupBidAmt'] = 0
r = requests.put(BASE_URL + uri, params={'fields': 'bidAmt'}, json=[retrieved_adkeyword], headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))


# 5. DELETE AdKeyword

uri = '/ncc/keywords/' + created_adkeyword['nccKeywordId']
method = 'DELETE'
r = requests.delete(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.content))


# Estimate Usage Sample

# 1. average-position-bid

uri = '/estimate/average-position-bid/keyword'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'device': 'PC', 'items': [{'key': '제주여행', 'position': 1}, {'key': '게스트하우스', 'position': 2}, {'key': '자전거여행', 'position': 3}]}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("#response status_code = {}".format(r.status_code))
print("#response body = {}".format(r.json()))


# 2. exposure-minimum-bid

uri = '/estimate/exposure-minimum-bid/keyword'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'device': 'PC', 'period': 'MONTH', 'items': ['제주여행', '게시트하우스', '자전거여행']}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# 3. median-bid

uri = '/estimate/median-bid/keyword'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'device': 'PC', 'period': 'MONTH', 'items': ['제주여행', '게시트하우스', '자전거여행']}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))


# 4. performance

uri = '/estimate/performance/keyword'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'device': 'PC', 'keywordplus': True, 'key': '중고차', 'bids': [100, 500, 1000, 1500, 2000, 3000, 5000]}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# 5. performance-bulk

uri = '/estimate/performance-bulk'
method = 'POST'
r = requests.post(BASE_URL + uri, json={'items': [{'device': 'PC', 'keywordplus': True, 'keyword': '제주여행', 'bid': 70}, {'device': 'PC', 'keywordplus': True, 'keyword': '제주도', 'bid': 80}, {'device': 'PC', 'keywordplus': True, 'keyword': '제주도맛집', 'bid': 90}, ]}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))

# Stat Usage Sample

# 1. GET Summary Report per multiple entities 

uri = '/stats'
method = 'GET'
stat_ids = [target_adgroup['nccCampaignId'], target_adgroup['nccAdgroupId']]
r = requests.get(BASE_URL + uri, params={'ids': stat_ids, 'fields': '["clkCnt","impCnt","salesAmt", "ctr", "cpc", "avgRnk", "ccnt"]', 'timeRange': '{"since":"2019-06-01","until":"2019-06-25"}'}, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

print("response status_code = {}".format(r.status_code))
print("response body = {}".format(r.json()))